In [ ]:
!pip install pyspark
from pyspark import SparkConf, SparkContext
from itertools import permutations
import collections
import csv
import time

conf = SparkConf().setMaster('local').setAppName('AprioriAlgorithm')
sc = SparkContext.getOrCreate(conf = conf)

In [ ]:
file = sc.textFile('store_data.csv')

start_time = time.time()

files = file.mapPartitions(lambda line: csv.reader(line))
items = file.flatMap(lambda line: line.split(',')) #[(A, B), (C)] -> [A, B, C]
#print(items.collect())
uniItems = items.distinct()
wrds = items.map(lambda word: (word, 1))
rdd = wrds.reduceByKey(lambda x,y: x+y)
print(rdd.collect())

minSupport = 150
supportRdd = rdd.filter(lambda x: x[1] >= minSupport)
baseRdd = supportRdd.map(lambda x: ([x[0]], x[1]))
print('1 table created')

supportRdd = supportRdd.map(lambda x: x[0])
rddCart = supportRdd

def removeDuplicatedItems(item): #(A,B) = (B,A), (c,c) -> (c)
  if (isinstance(item[0], tuple)):
    x = item[0]
    y = item[1]
  else:
    x = [item[0]]
    y = item[1]
  if (any(i == y for i in x) == False):
    temp = list(x)
    temp.append(y)
    temp.sort()
    result = tuple(temp)
    return result
  else:
    return x

combination_length = 2
while (supportRdd.isEmpty() == False):
  combined = supportRdd.cartesian(uniItems) #Combined with uniqueItems
  combined = combined.map(lambda item: removeDuplicatedItems(item))

  combined = combined.filter(lambda item: len(item) == combination_length)
  combined = combined.distinct() #Remove duplication

  combined2 = combined.cartesian(files)
  combined2 = combined2.filter(lambda item: all(x in item[1] for x in item[0]))

  combined2 = combined2.map(lambda item: item[0])
  combined2 = combined2.map(lambda item: (item, 1))
  combined2 = combined2.reduceByKey(lambda x,y: x+y)
  combined2 = combined2.filter(lambda item: item[1] >= minSupport)

  baseRdd = baseRdd.union(combined2)
  combined2 = combined2.map(lambda item: item[0])
  supportRdd = combined2
  
  combination_length +=1

  print(str(combination_length-1) +' table created')

class Filter():
  def __init__(self):
    self.stages = 1
  
  def calculateItemSet(self, a, b):
    if (len(a) > len(b)):
      return all(any(i == y for i in a) for y in b)
    else:
      return all(any(i == y for i in b) for y in a)

  def filterForConf(self, item, total): #tạo tập hợp riêng biệt từng cặp, mỗi cặp là 1 rule-
    if (len(item[0][0]) > len(item[1][0])):
      if (self.calculateItemSet(item[0][0], item[1][0]) == False):
        pass
      else:
        return (item)
    else:
      pass
    self.stages = self.stages + 1

  def calculateConfidence(self, item):
    parent = set(item[0][0])

    if (isinstance(item[1][0], str)):
      child = set([item[1][0]])
    else:
      child = set(item[1][0])

    pSup = item[0][1]
    chSup = item[1][1]
    sup = (pSup / chSup) 
    return list([list(child), list(parent.difference(child)), sup])

calculatedItems = baseRdd.cartesian(baseRdd)
fo = Filter()
total = calculatedItems.count()
rddConfidence = calculatedItems.filter(lambda item: fo.filterForConf(item, total))
rddConfidence = rddConfidence.map(lambda item: fo.calculateConfidence(item))
rddConfidence = rddConfidence.filter(lambda item: item[-1] >= 0.2)


print(rddConfidence.collect())
end_time = time.time()
print('Tong thoi gian: ', end_time-start_time)

[('shrimp', 536), ('almonds', 153), ('avocado', 250), ('vegetables mix', 193), ('green grapes', 68), ('whole weat flour', 70), ('yams', 86), ('cottage cheese', 239), ('energy drink', 200), ('tomato juice', 228), ('low fat yogurt', 574), ('green tea', 991), ('honey', 356), ('salad', 37), ('mineral water', 1788), ('salmon', 319), ('antioxydant juice', 67), ('frozen smoothie', 475), ('spinach', 53), ('olive oil', 494), ('burgers', 654), ('meatballs', 157), ('eggs', 1348), ('chutney', 31), ('turkey', 469), ('milk', 972), ('energy bar', 203), ('whole wheat rice', 439), ('whole wheat pasta', 221), ('french fries', 1282), ('soup', 379), ('light cream', 117), ('shallot', 58), ('frozen vegetables', 715), ('spaghetti', 1306), ('pet food', 49), ('cookies', 603), ('cooking oil', 383), ('champagne', 351), ('chocolate', 1230), ('chicken', 450), ('oil', 173), ('fresh tuna', 167), ('tomatoes', 513), ('black tea', 107), ('extra dark chocolate', 90), ('protein bar', 139), ('red wine', 211), ('pasta', 11